In [1]:
import requests
from bs4 import BeautifulSoup
import selenium
import pandas as pd
from selenium import webdriver
from datetime import datetime, date, time
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time
import re
import json

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
#options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(options=options)
driver.implicitly_wait(100)

In [3]:
url = "https://www.glassdoor.ie/Reviews/Aptiv-Reviews-E1891089_P2.htm"
driver.get(url)

In [4]:

html_soup = BeautifulSoup(driver.page_source, "html.parser")
containers = html_soup.find_all("div", class_="gdReview")

userEmail = "hassan.tariq@aptiv.com"
userPassword = "REKucuV4"


driver.find_element_by_xpath("/html/body/header/nav[1]/div/div/div/div[1]/button").click()
time.sleep(5)
driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[1]/div/div/input").send_keys(userEmail)
driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[2]/div/div/input").send_keys(userPassword)
driver.find_element_by_xpath("/html/body/div[11]/div/div/div[2]/div[2]/div[2]/div/div/div/div[3]/form/div[3]/div[1]/button").click()
time.sleep(5)
# read = driver.find_element_by_class_name("v2__EIReviewDetailsV2__continueReading v2__EIReviewDetailsV2__clickable v2__EIReviewDetailsV2__newUiCta mb")
# read.click()

for container in containers:

    # scraping review tile 
    review_title = container.find('a', class_="reviewLink")
    review_title = review_title.text
    print('Title: ', review_title)

    # scraping no of star reviews without subelements
    rating = container.find('span', class_='ratingNumber mr-xsm')
    rating = rating.text
    print('Rating: ', rating)

    # scrapign status of employee
    employee_Status = container.find('span', class_='pt-xsm pt-md-0 css-1qxtz39 eg4psks0')
    employee_Status = employee_Status.text
    print('Designation: ', employee_Status)

    
    rev_text = container.find('div', class_="v2__EIReviewDetailsV2__fullWidth")
    pros = rev_text.text
    pros = pros.strip('Pros')
    cons = rev_text.next_sibling.text
    cons = cons.strip('Cons')
    #advise = rev_text.next_sibling.next_sibling.text
    #advise = advise.strip('Advice to Management')
    print('pros: ', pros)
    print('cons: ', cons)
    #print('advise: ', advise)


    date_posted = container.find('span', class_='authorJobTitle middle common__EiReviewDetailsStyle__newGrey')
    date_posted = date_posted.text
    print('date & position: ', date_posted)

    author_info = container.find('span', class_='authorLocation')
    author_info = author_info.text
    print('Author Info: ', author_info)

    print()

Title:  Great Company with Progressive Technology
Rating:  5.0
Designation:  Current Employee, more than 1 year
pros:  Rare opportunity in Ireland to work in the automotive manufacturing industry. 
Genuine career opportunities.
The company truly embraces diversity.
Strong CEO and sense of being in the right place at the right time.
Salary, pension and health benefits above average but bonus is only average.
Good social scene, great offices and location, free breakfast, friendly co-workers and some memorable nights out.
cons:  Top-heavy management structure. Unfortunately there are too many middle management layers who only have administration/delegation capability rather than hands-on capability and direct experience.
date & position:  24 Nov 2019 - Anonymous Employee


AttributeError: 'NoneType' object has no attribute 'text'

In [9]:
Data = pd.DataFrame(
    {
        "Title": review_title,
        "Rating": rating,
        "Designation": employee_Status,
        "Pros": pros,
        "Cons": cons,
        "Author Info": author_info 
    }
    )   
Data.to_csv("{}.csv".format(review_title))
driver.quit()

ValueError: If using all scalar values, you must pass an index